## Installing packages

In [ ]:
!pip install --upgrade pip
!pip install "snowflake-connector-python[pandas]" "snowflake-snowpark-python[pandas]" snowflake-snowpark-python==1.9.0 numpy pandas matplotlib scikit-learn xgboost seaborn python-dateutil tqdm holidays faker
!pip install --upgrade --q snowflake-snowpark-python==1.9.0
!pip uninstall urllib3 -y
!pip install urllib3==1.26.15
!pip install fosforml==1.1.6

## Importing packages

In [11]:
from fosforml import *
from fosforml.constants import MLModelFlavours
from matplotlib import pyplot as plt
import pandas as pd
pd.set_option('display.max_columns', 500)
import seaborn as sns
from sklearn.metrics import mean_absolute_percentage_error
import numpy as np
import warnings; warnings.simplefilter('ignore')
from joblib import dump, load
import requests
from tqdm import tqdm
import time
import calendar

from time import sleep
import configparser
from dateutil.relativedelta import relativedelta
import datetime
from dateutil.easter import easter
from scipy.optimize import minimize_scalar
from scipy.optimize import curve_fit

%matplotlib inline

## Getting data from Github and moving to SF

In [2]:
# URL of the CSV file on GitHub
#url = 'https://github.com/aksh301091/fdc_akshaya_git/blob/91a0b3faf99492355d3816919fbf79de434926c3/ME_CASINO_PRJ/customer_table.csv'
#'https://raw.githubusercontent.com/username/repository/branch/filename.csv'

In [15]:
from fosforml.model_manager.snowflakesession import get_session
my_session = get_session()

In [12]:
# Read the CSV file into a DataFrame
cust_df = pd.read_csv('customer_table.csv')
#trx_df = pd.read_csv('transaction_table.csv')

#cust_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   PLAYER_ID                                10000 non-null  int64  
 1   AGE                                      10000 non-null  int64  
 2   GENDER                                   10000 non-null  object 
 3   HOME_COUNTRY                             10000 non-null  object 
 4   HOME_CITY                                10000 non-null  object 
 5   DATE_FIRST_VISIT                         10000 non-null  object 
 6   DATE_LAST_VISIT                          10000 non-null  object 
 7   TOTAL_NUMBER_OF_VISITS                   10000 non-null  int64  
 8   TOTAL_DURATION_SPENT                     10000 non-null  float64
 9   AVERAGE_DURATION_PER_VISIT               10000 non-null  float64
 10  TOTAL_CHIPS_WON_OR_LOST                  10000 

In [20]:
t1_df = pd.read_csv('trx_1.csv')
t2_df = pd.read_csv('trx_2.csv')
t2_df = pd.read_csv('trx_3.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'trx_1.csv'

In [14]:
type(cust_df)

pandas.core.frame.DataFrame

In [18]:
cust_sfdf = my_session.createDataFrame(cust_df)
cust_sfdf.write.mode("overwrite").save_as_table("casino_customers")
#my_session.table("casino_customers").show()

-------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"PLAYER_ID"  |"AGE"  |"GENDER"  |"HOME_COUNTRY"  |"HOME_CITY"  |"DATE_FIRST_VISIT"  |"DATE_LAST_VISIT"  |"TOTAL_NUMBER_OF_VISITS"  |"TOTAL_DURATION_SPENT"  |"AVERAGE_DURATION_PER_VISIT"  |"TOTAL_CHIPS_WON_OR_LOST"  |"AVERAGE_CHIPS_WON_OR_LOST_PER_VISIT"  |"UNIQUE_GAMES_PLAYED"  |"IS_PREMIUM_PLAYER"  |"IS_LOYALTY_CARD_HOLDER"  |"TOTAL_AMOUNT_SPENT_IN_HOTEL"  |"TOTAL

In [ ]:
for df in (t1_df, t2_df, t3_df):
    df =  df.append(df, ignore_index=True)
print df

In [19]:
#trx_sfdf = my_session.createDataFrame(trx_df)
#trx_sfdf.write.mode("overwrite").save_as_table("casino_transactions")

t1_sfdf = my_session.createDataFrame(t1_df)
t1_sfdf.write.mode("overwrite").save_as_table("casino_transactions")

t2_sfdf = my_session.createDataFrame(t2_df)
t2_sfdf.write.mode("overwrite").save_as_table("casino_transactions")

t3_sfdf = my_session.createDataFrame(t3_df)
t3_sfdf.write.mode("overwrite").save_as_table("casino_transactions")

#my_session.table("casino_transactions").show()

--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------
|"TRANSACTION_ID"  |"DATE"      |"PLAYER_ID"  |"PLAYER_AGE"  |"PLAYER_GENDER"  |"HOME_COUNTRY"  |"HOME_CITY"  |"GAME_CATEGORY"          |"GAME_NAME"       |"TABLE_MINIMUM_BET"  |"IS_PREMIUM_PLAYER"  |"DURATION_SPENT"  |"CHIPS_WON_OR_LOST"  |"NUMBER_OF_DAYS_SPENT_IN_CASINO"  |"STAYED_AT_CASINO_HOTEL"  |"AMOUNT_SPENT_IN_HOTEL_STAY"  |"NUMBER_OF_DAYS_SPENT_IN_HOTEL"  |"NUMBER_OF_RESTAURANT_VISITS"  |"AMOUNT_SPENT_I

In [10]:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 25 columns):
 #   Column                                   Non-Null Count  Dtype  
---  ------                                   --------------  -----  
 0   PLAYER_ID                                10000 non-null  int64  
 1   AGE                                      10000 non-null  int64  
 2   GENDER                                   10000 non-null  object 
 3   HOME_COUNTRY                             10000 non-null  object 
 4   HOME_CITY                                10000 non-null  object 
 5   DATE_FIRST_VISIT                         10000 non-null  object 
 6   DATE_LAST_VISIT                          10000 non-null  object 
 7   TOTAL_NUMBER_OF_VISITS                   10000 non-null  int64  
 8   TOTAL_DURATION_SPENT                     10000 non-null  float64
 9   AVERAGE_DURATION_PER_VISIT               10000 non-null  float64
 10  TOTAL_CHIPS_WON_OR_LOST                  10000 

In [ ]:
## Reading data from SF 
table_name = 'DF_OPT'
df_opt = my_session.sql("select * from {}".format(table_name))
df_opt = df_opt.to_pandas()